# Introduction

**Overview:** Brief description of the problem, the dataset, and the main objectives of the project.

# Setup 

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
import pickle
import os

## Environment Variables 
**Note**: Setting environment variables is optional, but it is recommended if you store sensitive information (such as API keys or database credentials) in a `.env` file. Using environment variables helps keep such information secure and separate from your codebase.

In [ ]:
# Imports
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API key from .env 
api_key = os.getenv("API_KEY")

# Get SQL database credentials from .env
sql_username = os.getenv("SQL_USERNAME")
sql_password = os.getenv("SQL_PASSWORD")

# Data Loading

## CSV 

In [ ]:
# Load data from a csv file into a Pandas DataFrame
df = pd.read_csv("your_csv_file_here.csv")

## MySQL

In [ ]:
# Imports 
from sqlalchemy import create_engine

# Database info
mysql_host = "localhost"  # Default hostname for a MySQL server running locally
mysql_port = 3306  # Default port for MySQL
mysql_database_name = "your_mysql_database_name_here"
mysql_table_name = "your_mysql_table_name_here"

# Create an SQLAlchemy engine for interacting with the MySQL database
engine = create_engine(f"mysql+mysqlconnector://{sql_username}:{sql_password}@{mysql_host}:{mysql_port}/{mysql_database_name}")

# Load data from MySQL database into a Pandas DataFrame
with engine.connect() as connection:
    df = pd.read_sql(f"SELECT * FROM {mysql_table_name}", con=connection)

# Initial Data Inspection  
Basic exploration of the dataset to understand its structure and detect obvious issues.

In [ ]:
# Show DataFrame info to check the number of rows and columns, data types and missing values
df.info()

In [ ]:
# Show top five rows to get a sense of the data
df.head()

# Data Preprocessing

## Handling Duplicates

Duplicates based on all columns:

In [ ]:
# Diagnose duplicates 
df.duplicated().value_counts()

In [ ]:
# Remove duplicates
df = df.drop_duplicates().copy()

Duplicates based on specific columns, e.g. the ID column or a combination of columns:

In [ ]:
# Diagnose duplicates
df.duplicated(subset=["column_1", "column_2", "column_3"]).value_counts()

In [ ]:
# Remove duplicates
df = df.drop_duplicates(subset=["column_1", "column_2", "column_3"]).copy()

## Handling Incorrect Data Types

In [ ]:
# Convert column from str to int
df["int_column"] = df["str_column"].astype("Int32")

In [ ]:
# Convert column from str to datetime
df["datetime_column"] = pd.to_datetime(df["str_column"])

## Feature Extraction

### Categorical from String  
Extract a categorical feature from a string column.

In [ ]:
# Function to extract a category from a string   
def extract_category_from_string(string):
    # Map categories to their corresponding list of keywords
    category_keywords_map = {
        "Category 1": ["Keyword 1", "Keyword 2", "Keyword 3"],
        "Category 2": ["Keyword 4", "Keyword 5", "Keyword 6"],
        "Category 3": ["Keyword 7", "Keyword 8", "Keyword 9"]
    }

    # Loop through each category and its associated keywords 
    for category, keywords in category_keywords_map.items():
        # Check if any keyword is present in the string
        if any(keyword in string for keyword in keywords):
            return category  # Return the category corresponding to the keyword
    return np.nan  # Return a missing value if no keyword matches

# Apply function on an existing string column to create a new categorical feature column
df["categorical_feature"] = df["str_column"].apply(extract_category_from_string)

# Show category frequencies
print(df["categorical_feature"].value_counts())

### Numerical from String  
Extract a numerical feature from a string column.

In [ ]:
# Imports
import re

# Function to extract the first number in a string 
def extract_number_from_string(string):
    first_number = re.search(r"\b-?\d+([.,]\d+)?\b", string)  # searches for first integer or float (positive or negative; decimal separator "." or ",")
    if first_number:
        return float(first_number.group().replace(",", "."))  # Replace "," with "." as decimal separator  
    else:
        return np.nan  # Return a missing value if no number in string

# Apply function on an existing string column to create a new numerical feature column
df["numerical_feature"] = df["str_column"].apply(extract_number_from_string)

# Show descriptive statistics of numerical feature
df["numerical_feature"].describe()

### Boolean from String  
Extract a boolean feature from a string column.

In [ ]:
# List of keywords to determine if the feature is present or absent
keywords = ["keyword 1", "keyword 2", "keyword 3"]

# Extract boolean feature column: True if any keyword is found in the string column
df["boolean_feature"] = df["str_column"].apply(lambda x: any(keyword.lower() in x.lower() for keyword in keywords))

# Show frequencies of boolean feature
print(df["boolean_feature"].value_counts())

## Handling Missing Values

### Imputation

Imputation for numerical column:

In [ ]:
# Descriptive statistics of numerical column
df["numerical_column"].describe()

In [ ]:
# Impute missing values with the median
median = df["numerical_column"].median()
df["numerical_column"] = df["numerical_column"].fillna(median)

Imputation for categorical column:

In [ ]:
# Frequencies of categorical column
df["categorical_column"].value_counts()

In [ ]:
# Impute missing values with the mode 
mode = df["categorical_column"].mode()[0]
df["categorical_column"] = df["categorical_column"].fillna(mode)

### Deletion

In [ ]:
# Delete rows where any column has a missing value 
df.dropna(inplace=True)

In [ ]:
# Delete rows where either column_1 or column_2 has a missing value 
df.dropna(subset=["column_1", "column_2"], how="any", inplace=True)

## Handling Outliers

### Remove with 3SD Method  
Remove univariate outliers from a numerical column by applying the 3 standard deviation (SD) rule. Specifically, a data point is considered an outlier if it falls more than 3 standard deviations above or below the mean of the column.  

In [ ]:
# Create a custom transformer class to remove outliers using the 3SD method
class OutlierRemover3SD(BaseEstimator, TransformerMixin):
    def fit(self, df, numerical_column):
        # Calculate mean, standard deviation, and cutoff values of the numerical column
        self.mean_ = df[numerical_column].mean()
        self.sd_ = df[numerical_column].std()
        self.lower_cutoff_ = self.mean_ - 3 * self.sd_
        self.upper_cutoff_ = self.mean_ + 3 * self.sd_

        # Create a mask for filtering outliers
        self.mask_ = (df[numerical_column] >= self.lower_cutoff_) & (df[numerical_column] <= self.upper_cutoff_)

        # Show cutoff values and number of outliers
        print(f"Lower cutoff for {numerical_column}: {self.lower_cutoff_}")
        print(f"Upper cutoff for {numerical_column}: {self.upper_cutoff_}")
        print(f"{df.shape[0] - df[self.mask_].shape[0]} outliers removed based on {numerical_column} using the 3SD method.")
  
        return self

    def transform(self, df):
        # Remove outliers based on the mask 
        return df[self.mask_]

    def fit_transform(self, df, numerical_column):
        # Perform both fit and transform 
        return self.fit(df, numerical_column).transform(df)

In [ ]:
# Initialize outlier remover 
outlier_remover_3sd = OutlierRemover3SD()

# Remove outliers
df = outlier_remover_3sd.fit_transform(df, "numerical_column")

### Remove with 1.5 IQR Method  
Remove univariate outliers from a numerical column using the 1.5 interquartile range (IQR) rule. Specifically, a data point is considered an outlier if it falls more than 1.5 interquartile ranges above the third quartile (Q3) or below the first quartile (Q1) of the column.  

In [ ]:
# Create a custom transformer class to remove outliers using the 1.5 IQR method
class OutlierRemoverIQR(BaseEstimator, TransformerMixin):
    def fit(self, df, numerical_column):
        # Calculate quartiles, IQR and cutoff values 
        Q1 = df[numerical_column].quantile(0.25)
        Q3 = df[numerical_column].quantile(0.75)
        IQR = Q3 - Q1
        self.lower_cutoff_ = Q1 - 1.5 * IQR
        self.upper_cutoff_ = Q3 + 1.5 * IQR

        # Create a mask for filtering outliers
        self.mask_ = (df[numerical_column] >= self.lower_cutoff_) & (df[numerical_column] <= self.upper_cutoff_)
  
        # Show cutoff values and number of outliers
        print(f"Lower cutoff for {numerical_column}: {self.lower_cutoff_}")
        print(f"Upper cutoff for {numerical_column}: {self.upper_cutoff_}")
        print(f"{df.shape[0] - df[self.mask_].shape[0]} outliers removed based on {numerical_column} using the 1.5 IQR method.")
        
        return self

    def transform(self, df):
        # Remove outliers based on the mask 
        return df[self.mask_]

    def fit_transform(self, df, numerical_column):
        # Perform both fit and transform
        return self.fit(df, numerical_column).transform(df)

In [ ]:
# Initialize outlier remover 
outlier_remover_iqr = OutlierRemoverIQR()

# Remove outliers
df = outlier_remover_iqr.fit_transform(df, "numerical_column")

## Saving Data

### CSV  
Save preprocessed data from a Pandas DataFrame to a `csv` file in the `data` directory.

In [ ]:
# Create data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Save as csv  
df.to_csv("data/preprocessed_data.csv", index=False)

### MySQL  
Save preprocessed data from a Pandas DataFrame to a MySQL table.  
Note: Make sure `sql_username` and `sql_password` were imported as environment variables.

In [ ]:
# Imports 
from sqlalchemy import create_engine

# Database info
mysql_host = "localhost"  # Default hostname for a MySQL server running locally
mysql_port = 3306  # Default port for MySQL
mysql_database_name = "your_mysql_database_name_here"
mysql_table_name = "preprocessed_data"

try:
    # Create an SQLAlchemy engine for interacting with the MySQL database
    engine = create_engine(f"mysql+mysqlconnector://{sql_username}:{sql_password}@{mysql_host}:{mysql_port}/{mysql_database_name}")
    
    # Save data to MySQL 
    with engine.connect() as connection:
        df.to_sql(name=mysql_table_name, con=connection, if_exists="replace", index=False)
    
    print("Preprocessed data successfully saved to MySQL.")

except Exception as e:
    print(f"Error saving preprocessed data to MySQL: {e}")

# Exploratory Data Analysis (EDA)

## Defining Column Types  
Define numerical, categorical and boolean columns for downstream tasks like exploratory data analysis, additional preprocessing steps, and machine learning.

### Manual  
Option 1: Use the manual approach for small datasets or when you have specific requirements and need precise control.

In [ ]:
# Show columns and their pandas data types
df.info()

In [ ]:
# Define column types manually
numerical_columns = ["numerical_column_1", "numerical_column_2", "numerical_column_3"]
categorical_columns = ["categorical_column_1", "categorical_column_2", "categorical_column_3"]
boolean_columns = ["boolean_column_1", "boolean_column_2", "boolean_column_3"]

### Programmatic  
Option 2: Use the programmatic approach for large datasets or when you need to automate the process, ensuring efficiency and scalability in handling column types.

In [ ]:
# Define column types programmatically based on pandas data types
numerical_columns = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_columns = df.select_dtypes(include=["object"]).columns.tolist()
boolean_columns = df.select_dtypes(include=["bool"]).columns.tolist()

## Univariate EDA  
Univariate exploratory data analysis (EDA) analyzes the distribution, central tendency, and variability of a single column using descriptive statistics and visualizations.

### Numerical Columns  
Perform univariate EDA on numerical columns by examining descriptive statistics (such as mean, median, standard deviation and quartiles) and histograms for distributions.

#### Descriptive Statistics

Single Column:

In [ ]:
# Descriptive statistics of a single numerical column
df["numerical_column"].describe()

Multiple Columns:

In [ ]:
# Table with descriptive statistics of all numerical columns
df[numerical_columns].describe().transpose()

#### Histogram 

Single Column:

In [ ]:
# Create histogram of numerical column
sns.histplot(df["numerical_column"])

# Add title and axes labels 
plt.title("Distribution of numerical_column")
plt.xlabel("numerical_column")
plt.ylabel("Frequency")

# Show the plot
plt.show()

### Categorical Columns  
Perform univariate EDA on categorical columns by examining descriptive statistics (such as absolute and relative frequencies) and bar plots of frequencies.

#### Frequencies

Single Column:

Absolute and relative frequencies of a single categorical colum:

In [ ]:
# Calculate absolute and relative frequencies 
absolute_frequencies = df["categorical_column"].value_counts()
relative_frequencies = absolute_frequencies / absolute_frequencies.sum() * 100

# Show frequencies
print(f"Absolute frequencies:\n {absolute_frequencies}\n")
print(f"Relative frequencies:\n {relative_frequencies.round(1)}")

Bar plot of frequencies of a single categorical colum:

In [ ]:
# Bar plot
sns.barplot(x=absolute_frequencies.index, y=absolute_frequencies.values, palette="colorblind")

# Add title and axes labels 
plt.title("categorical_column")
plt.ylabel("Frequency")

# Rotate x-axis tick labels by 45 degrees
plt.xticks(rotation=45)

# Show the plot
plt.show()

Multiple Columns:

Absolute and relative frequencies of multiple categorical colums:

In [ ]:
# Initialize dictionaries to store frequencies for multiple categorical columns
absolute_frequencies_dict = {}
relative_frequencies_dict = {}

# Iterate over the categorical columns
for categorical_column in categorical_columns:
    # Calculate absolute and relative frequencies 
    absolute_frequencies = df[categorical_column].value_counts()
    relative_frequencies = absolute_frequencies / absolute_frequencies.sum() * 100

    # Store frequencies
    absolute_frequencies_dict[categorical_column] = absolute_frequencies
    relative_frequencies_dict[categorical_column] = relative_frequencies

    # Show frequencies
    print(categorical_column.upper())
    print(f"Absolute frequencies:\n {absolute_frequencies}\n")
    print(f"Relative frequencies (%):\n {relative_frequencies.round(1)}")
    print("=" * 50 + "\n")

Bar plot matrix that shows frequencies for multiple categorical columns.  
Example code for 5 bar plots in a 2x3 matrix:

In [ ]:
# Set the figure size
plt.figure(figsize=(12, 6))

# Iterate over the categorical columns
for i, categorical_column in enumerate(categorical_columns):
    # Create a subplot in a 2x3 grid (current subplot i+1 because subplot indices start at 1)
    plt.subplot(2, 3, i + 1)
    
    # Calculate frequencies for the current column
    absolute_frequencies = df[categorical_column].value_counts()
    
    # Create bar plot for the current column
    sns.barplot(x=absolute_frequencies.index, y=absolute_frequencies.values, estimator=np.median, ci=None)
    
    # Add title and axes labels
    plt.title(categorical_column.title())
    plt.ylabel("Frequency")

    # Rotate x-axis tick labels by 45 degrees
    plt.xticks(rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

## Bivariate EDA  
Bivariate exploratory data analysis (EDA) examines relationships between two columns using methods such as correlation analysis, scatterplots, group-wise statistics, and bar plots.

### Numerical with Numerical  
Analyze relationships between two numerical columns using correlation methods and visualizations like scatterplots. 

#### Correlations

Correlation matrix:

In [ ]:
# Combine numerical and boolean columns for correlation analysis
combined_columns = numerical_columns + boolean_columns

# Calculate the correlation matrix 
correlation_matrix = df[combined_columns].corr()

# Round correlations to 2 decimals
correlation_matrix = round(correlation_matrix, 2)

# Create an upper triangle mask (k=1 excludes the diagonal)
mask = np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)

# Set upper triangle to NaN to avoid redundant information
correlation_matrix[mask] = np.nan

# Show correlation matrix
correlation_matrix

Correlation heatmap:

In [ ]:
# Set the figure size
plt.figure(figsize=(8, 6))

# Create heatmap
sns.heatmap(
    correlation_matrix, 
    cmap="viridis",  # Color map choice
    annot=True,  # Show numbers
    linewidth=0.5  # Thin white lines between cells
)

# Add title
plt.title("Correlation Heatmap")

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

Save correlation heatmap as an image:

In [ ]:
# Imports
import os

# Create "images" directory if it doesn't exist
os.makedirs("images", exist_ok=True)

# Save the heatmap as a .png image
try:
    # Construct full file path
    image_path = os.path.join("images", "correlation_heatmap.png") 
    
    # Save the heatmap  
    plt.savefig(
        image_path, 
        bbox_inches="tight",  # removes unnecessary whitespace
        dpi=300  # higher image quality
    )
    print(f"Correlation heatmap saved successfully to {image_path}")

except Exception as e:
    print(f"Error saving correlation heatmap: {e}")

Correlations of target variable with features:

In [ ]:
# Correlations between the numerical target variable and each numerical and boolean feature
combined_columns = numerical_columns + boolean_columns
df[combined_columns].corr()["numerical_target"].sort_values(ascending=False)

#### Scatterplot

In [ ]:
# Set the figure size
plt.figure(figsize=(8, 6))

# Create the scatterplot 
sns.scatterplot(data=df, x="numerical_column_1", y="numerical_column_2")

# Add title and axis labels
plt.title("Relationship between numerical_column_1 and numerical_column_2")
plt.xlabel("numerical_column_1")
plt.ylabel("numerical_column_2")

# Show the plot
plt.show()

#### Scatterplot Matrix 
Scatterplot matrix that shows scatterplots between a numerical column (e.g., the target variable) and each other numerical column.   
Example code for 9 scatterplots in a 3x3 matrix:

In [ ]:
# Set the figure size 
plt.figure(figsize=(12, 12))

# List of numerical columns excluding the target column
numerical_columns_without_target = [col for col in numerical_columns if col != "numerical_target"]

# Iterate over the numerical columns
for i, numerical_column in enumerate(numerical_columns_without_target):
    # Create a subplot in a 3x3 grid (current subplot i+1 because subplot indices start at 1)
    plt.subplot(3, 3, i + 1)
    
    # Create a scatterplot between the current column and the numerical target
    sns.scatterplot(data=df, x=numerical_column, y="numerical_target")
    
    # Add title and axis labels 
    plt.title(f"numerical_target by {numerical_column}")
    plt.xlabel(f"{numerical_column}")
    plt.ylabel("numerical_target")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

### Numerical with Categorical  
Examine relationships between a numerical column and a categorical column using group-wise statistics (such as median or mean by category), and visualizations like bar plots to compare statistics by category.

#### Group-Wise Statistics 
Descriptive statistics of a numerical column grouped by categories in a categorical column.

In [ ]:
# Descriptive statistics of numerical column by categorical column
numerical_by_category = df["numerical_column"].groupby(df["categorical_column"])
numerical_by_category.describe()

#### Bar Plot  
Bar plot of the median of a numerical column by a single categorical column.

In [ ]:
# Store the median of the numerical column by category
median_by_category = numerical_by_category.median()

# Bar plot of median by category
sns.barplot(x=median_by_category.index, y=median_by_category.values, palette="colorblind")

# Add title and axes labels
plt.title("Median numerical_column by categorical_column")
plt.xlabel("Category")
plt.ylabel("numerical_column")

# Show the plot
plt.show()

#### Bar Plot Matrix  
Bar plot matrix that shows bar plots between a single numerical column (e.g., the target variable) and each categorical column.  
Example code for 5 bar plots in a 2x3 matrix:

In [ ]:
# Set the figure size to 12x6 inches
plt.figure(figsize=(12, 6))

# Iterate over the categorical columns
for i, categorical_column in enumerate(categorical_columns):
    # Create a subplot in a 2x3 grid (current subplot i+1 because subplot indices start at 1)
    plt.subplot(2, 3, i + 1)
    
    # Create a bar plot of the median of the numerical column by the current categorical column
    ax = sns.barplot(data=df, x=categorical_column, y="numerical_column", estimator=np.median, ci=None)
    
    # Add title and axes labels
    plt.title(f"Median numerical_column by {categorical_column}")
    plt.xlabel("Category")
    plt.ylabel("numerical_column")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

# Train-Validation-Test Split  
The ideal split depends on the dataset size and the task. A general guideline is:

| Dataset Size                | Training Set | Validation Set | Test Set |
|:-----------------------------|--------------|----------------|----------|
| Smaller datasets (<10,000)   | 70%          | 15%            | 15%      |
| Larger datasets (≥10,000)    | 80%          | 10%            | 10%      | 

In [ ]:
# Split the data into X features and y target
X = df.drop("target", axis=1)
y = df["target"]

In [ ]:
# Split the data into training and temporary sets (70% train, 30% temporary)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temporary data into validation and test sets (50% each)
# Note: This accomplishes a 70% training, 15% validation and 15% test set size
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Delete the temporary data to free up memory
del X_temp, y_temp

# Feature Scaling and Encoding  
Use a `ColumnTransformer` to preprocess columns based on their data type. This allows the appropriate transformation to each column type in a single pipeline:  
- Scale numerical columns: Use `StandardScaler` (scales features to have mean = 0 and standard deviation = 1) or optionally `MinMaxScaler` (scales features to a range between [0, 1]).
- Encode categorical columns: Apply `OneHotEncoder` to convert categorical variables into binary (one-hot) encoded vectors.
- Pass through boolean columns unchanged: Retain boolean features in their original form using `remainder="passthrough"`.

In [ ]:
# Initialize a column transformer 
column_transformer = ColumnTransformer(
    transformers=[
        ("scaler", StandardScaler(), numerical_columns),  # Use MinMaxScaler() for min-max normalization
        ("encoder", OneHotEncoder(drop="first"), categorical_columns)  
    ],
    remainder="passthrough" 
)

# Fit the column transformer to the training data and apply transformations
X_train_transformed = column_transformer.fit_transform(X_train)

# Apply the same transformations to the validation and test data
X_val_transformed = column_transformer.transform(X_val)
X_test_transformed = column_transformer.transform(X_test)

# Modeling (Regression Task)  
For a regression problem, where the goal is to predict a numerical target variable.

## Helper Function for Residual Plots  
Creates two plots for predicted vs. actual target and residuals vs. actual target to evaluate model performance and identify potential issues in the model assumptions graphically.

In [ ]:
# Helper function to create residual plots
def plot_residuals(y, y_pred):
    # Calculate residuals
    residuals = [actual_value - predicted_value for actual_value, predicted_value in zip(y, y_pred)]

    # Create a 1x2 grid of subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), dpi=150)

    # Plot 1: Predicted vs. Actual Target
    axes[0].scatter(y, y_pred)
    axes[0].plot([min(y), max(y)], 
                 [min(y), max(y)], 
                 color="red", 
                 linestyle="--", 
                 label="Perfect Prediction")  # Add diagonal reference line
    axes[0].set_title("Predicted vs. Actual Target")
    axes[0].set_xlabel("Actual Target")
    axes[0].set_ylabel("Predicted Target")
    axes[0].grid(True)
    axes[0].legend() 

    # Plot 2: Residuals vs. Actual Target
    axes[1].scatter(y, residuals)
    axes[1].axhline(y=0, color="red", linestyle="--", label="Perfect Prediction")  # Add horizontal reference line
    axes[1].set_xlabel("Actual Target")
    axes[1].set_ylabel("Residuals")
    axes[1].set_title("Residuals vs. Actual Target")
    axes[1].grid(True)
    axes[1].legend() 

    # Adjust layout and display the plots
    plt.tight_layout()
    plt.show()

## Training Baseline Models (Individually)  

### Linear Regression  
Hyperparameters and Default Values:
- `fit_intercept=True`: Calculates the intercept; can be set to `False` if data is already centered.
- `n_jobs=None`: Number of CPU threads; use `-1` for all available processors.
- `positive=False`: Forces regression coefficients to be non-negative if set to `True`.

For more details, refer to the official [scikit-learn LinearRegression documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression).  

In [ ]:
# Initialize model
lr = LinearRegression()

# Train model
lr.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = lr.predict(X_train_transformed)
y_val_pred = lr.predict(X_val_transformed)

# Evaluate model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Create table of evaluation metrics
lr_evaluation = pd.DataFrame({
    "Metric": ["RMSE", "MAPE", "R-squared"],
    "Training": [train_rmse, train_mape, train_r2],
    "Validation": [val_rmse, val_mape, val_r2]
})

# Show evaluation metrics
print(lr_evaluation.round(2))  # round metrics to 2 decimals

### Support Vector Regressor  
Hyperparameters and Default Values:
- Model Complexity:
    - `C=1.0`: Regularization parameter balancing error reduction and model complexity.
    - `epsilon=0.1`: Margin of tolerance for predictions without penalty.
- Kernel Configuration:
    - `kernel="rbf"`: Kernel function for mapping data to higher dimensions (default is radial basis function or "rbf"). 
    - `degree=3`: Degree of the polynomial kernel function (ignored by the rbf kernel). 
    - `gamma="scale"`: Influence range of a single training example. `"scale"` means `1 / (n_features * X.var())`.
    - `coef0=0.0`: Independent term in polynomial and sigmoid kernel function (ignored by the rbf kernel).
- Training Behavior:
    - `tol=1e-3`: Stopping criterion for optimization. If the change in the cost function is less than this tolerance, training stops.
    - `cache_size=200`: Memory (MB) allocated for kernel computation. Larger values speed up training.
    - `shrinking=True`: Enables the shrinking heuristic, which can speed up training by eliminating unnecessary steps during optimization.
    - `verbose=False`: Whether to print progress messages during training.
    - `max_iter=-1`: Maximum number of iterations during training (`-1` for no limit).

For more details, refer to the official [scikit-learn SVR documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR).  

In [ ]:
# Initialize model
svr = SVR()

# Train model
svr.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = svr.predict(X_train_transformed)
y_val_pred = svr.predict(X_val_transformed)

# Evaluate model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Create table of evaluation metrics
svr_evaluation = pd.DataFrame({
    "Metric": ["RMSE", "MAPE", "R-squared"],
    "Training": [train_rmse, train_mape, train_r2],
    "Validation": [val_rmse, val_mape, val_r2]
})

# Show evaluation metrics
print(svr_evaluation.round(2))  # round metrics to 2 decimals

### Random Forest Regressor  
Hyperparameters and Default Values:
- Model Complexity:
  - `n_estimators=100`: Number of trees in the forest.  
  - `max_depth=None`: Maximum depth of each tree; `None` allows trees to grow until all leaves are pure or minimum samples are reached.  
  - `min_samples_split=2`: Minimum number of samples required to split a node.  
  - `min_samples_leaf=1`: Minimum number of samples required at a leaf node.  
  - `max_features='auto'`: Number of features considered for the best split; default `auto` uses the square root of all features.  
- Regularization:
  - `max_leaf_nodes=None`: Maximum number of leaf nodes per tree.  
  - `min_impurity_decrease=0.0`: Splits a node only if it decreases impurity by this threshold.  
- Training Behavior:
  - `bootstrap=True`: Whether to use bootstrap samples for training each tree.  
  - `oob_score=False`: Whether to use out-of-bag samples to estimate prediction accuracy.  
  - `n_jobs=None`: Number of CPU threads used (`-1` for all processors).  
  - `random_state=None`: Random seed for reproducibility.  
  - `verbose=0`: Controls the verbosity of output during training.  
- Performance Optimization:
  - `max_samples=None`: Maximum number of samples used to train each tree, useful for subsampling large datasets.
 
For more details, refer to the official [scikit-learn RandomForestRegressor documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor).  

In [ ]:
# Initialize model
rf = RandomForestRegressor(random_state=42)

# Train model
rf.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = rf.predict(X_train_transformed)
y_val_pred = rf.predict(X_val_transformed)

# Evaluate model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Create table of evaluation metrics
rf_evaluation = pd.DataFrame({
    "Metric": ["RMSE", "MAPE", "R-squared"],
    "Training": [train_rmse, train_mape, train_r2],
    "Validation": [val_rmse, val_mape, val_r2]
})

# Show evaluation metrics
print(rf_evaluation.round(2))  # round metrics to 2 decimals

### Multi-Layer Perceptron Regressor  
Hyperparameters and Default Values:
- Model Architecture:
    - `hidden_layer_sizes=(100,)`: Defines the size and number of hidden layers; `(100,)` indicates one layer with 100 neurons.  
    - `activation="relu"`: Activation function for the hidden layers; options include `"relu"`, `"tanh"`, `"logistic"`, or `"identity"`.  
    - `solver="adam"`: Optimization algorithm; options are `"adam"` (default), `"lbfgs"`, or `"sgd"`.  
- Regularization and Learning:  
    - `alpha=0.0001`: L2 regularization term to prevent overfitting.  
    - `learning_rate="constant"`: Strategy for learning rate adjustment; options are `"constant"`, `"invscaling"`, or `"adaptive"`.  
    - `learning_rate_init=0.001`: Initial learning rate for weight updates.  
    - `power_t=0.5`: Exponent for inverse scaling of learning rate (used when `learning_rate="invscaling"`).  
- Training Behavior:  
    - `max_iter=200`: Maximum number of iterations for training.  
    - `tol=1e-4`: Tolerance for stopping criteria; training stops if loss improvement is below this value.  
    - `momentum=0.9`: Momentum parameter for gradient descent updates (used when `solver="sgd"`).  
    - `n_iter_no_change=10`: Number of iterations with no improvement to stop early.  
    - `early_stopping=False`: Enables early stopping when validation score doesn’t improve.  
- Performance Optimization:  
    - `batch_size="auto"`: Number of samples per batch for training; `"auto"` uses `min(200, n_samples)`.  
    - `shuffle=True`: Whether to shuffle training data before each epoch.  
    - `random_state=None`: Random seed for reproducibility.  
    - `verbose=False`: Controls verbosity of output during training.  
    - `warm_start=False`: Reuses previous solution to initialize weights for additional fitting.  
    - `beta_1=0.9`, `beta_2=0.999`: Exponential decay rates for moving averages of gradients and squared gradients (used in `solver="adam"`).  
    - `epsilon=1e-8`: Small value to prevent division by zero in `solver="adam"`.

For more details, refer to the official [scikit-learn MLPRegressor documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html).

In [ ]:
# Initialize model
mlp = MLPRegressor(random_state=42)

# Train model
mlp.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = mlp.predict(X_train_transformed)
y_val_pred = mlp.predict(X_val_transformed)

# Evaluate model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Create table of evaluation metrics
mlp_evaluation = pd.DataFrame({
    "Metric": ["RMSE", "MAPE", "R-squared"],
    "Training": [train_rmse, train_mape, train_r2],
    "Validation": [val_rmse, val_mape, val_r2]
})

# Show evaluation metrics
print(mlp_evaluation.round(2))  # round metrics to 2 decimals

### XGBoost Regressor  
Hyperparameters and Default Values:
- Model Complexity:  
    - `n_estimators=100`: Number of trees.  
    - `max_depth=6`: Maximum depth of each tree.  
    - `learning_rate=0.3`: Step size shrinking to prevent overfitting.  
    - `subsample=1.0`: Fraction of training samples used for each tree.  
    - `colsample_bytree=1.0`: Fraction of features used for each tree.  
    - `colsample_bylevel=1.0`: Fraction of features used at each tree level.  
    - `colsample_bynode=1.0`: Fraction of features used at each node.  
- Regularization and Learning:  
    - `gamma=0`: Minimum loss reduction required to make a further partition on a leaf node.  
    - `min_child_weight=1`: Minimum sum of instance weight (hessian) in a child.  
    - `scale_pos_weight=1`: Controls the balance of positive and negative weights; used for imbalanced datasets.  
- Training Behavior:  
    - `objective="reg:squarederror"`: Objective function for regression; default is for squared error.  
    - `booster="gbtree"`: Booster type; options include `"gbtree"` (default), `"gblinear"`, and `"dart"`.  
    - `tree_method="auto"`: Tree construction algorithm; `"auto"` chooses based on system configuration. Options include `"exact"`, `"approx"`, and `"hist"`.  
    - `eval_metric="rmse"`: Metric used for validation during training; default is root mean square error (`rmse`).  
- Performance Optimization:  
    - `early_stopping_rounds=None`: Stops training if validation metric does not improve after specified rounds.  
    - `n_jobs=1`: Number of threads used for parallel computation (`-1` for all processors).  
    - `random_state=None`: Seed for reproducibility.  
    - `verbose=1`: Verbosity level for training output; 0 for silent, higher values show more details.  
- Advanced Parameters:  
    - `lambda=1`: L2 regularization term on weights.  
    - `alpha=0`: L1 regularization term on weights.  
    - `max_delta_step=0`: Used to help with convergence in highly imbalanced datasets.

For more details, refer to the official [XGBoost documentation](https://xgboost.readthedocs.io/en/latest/parameter.html).

In [ ]:
# Initialize model
xgb = XGBRegressor(random_state=42)

# Train model
xgb.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = xgb.predict(X_train_transformed)
y_val_pred = xgb.predict(X_val_transformed)

# Evaluate model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Create table of evaluation metrics
xgb_evaluation = pd.DataFrame({
    "Metric": ["RMSE", "MAPE", "R-squared"],
    "Training": [train_rmse, train_mape, train_r2],
    "Validation": [val_rmse, val_mape, val_r2]
})

# Show evaluation metrics
print(xgb_evaluation.round(2))  # round metrics to 2 decimals

## Training Baseline Models (Pipeline)  
Train five baseline models:
- Linear Regression
- Support Vector Regressor
- Random Forest Regressor
- Multi-Layer Perceptron Regressor
- XGBoost Regressor

The model performance will be evaluated using the following metrics:  
- Root Mean Squared Error (RMSE)
- Mean Absolute Percentage Error (MAPE)
- R-squared (R2)

In [ ]:
# Define models with baseline configurations
models = [LinearRegression(), SVR(), RandomForestRegressor(random_state=42), MLPRegressor(random_state=42), XGBRegressor(random_state=42)]

# Create lists for storing the evaluation metrics (RMSE, MAPE, R2) of each model 
rmse_ls = []
mape_ls = []
r2_ls = []

# Loop through each model
for model in models:
    # Show model
    print("=" * 100)
    print(f"Model: {model}")

    # Scale numerical columns and encode categorical columns 
    column_transformer = ColumnTransformer(
        transformers=[
            ("scaler", StandardScaler(), numerical_columns),
            ("encoder", OneHotEncoder(drop="first"), categorical_columns)
        ],
        remainder="passthrough"  # Include the boolean columns without transformation
    )

    # Create a pipeline
    pipeline = Pipeline(steps=[
        ("column_transformer", column_transformer),
        ("model", model)
    ])

    # Fit the pipeline on the training data
    pipeline.fit(X_train, y_train)
    
    # Predict on the validation data
    y_val_pred = pipeline.predict(X_val)

    # Calculate evaluation metrics: RMSE, MAPE, R2
    rmse = mean_squared_error(y_val, y_val_pred, squared=False)
    mape = mean_absolute_percentage_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)

    # Show evaluation metrics
    print(f"RMSE: {round(rmse, 2)}")
    print(f"MAPE: {round(mape, 2)}")
    print(f"R-squared (R²): {round(r2, 2)}")

    # Add evaluation metrics to their respective lists
    rmse_ls.append(rmse)
    mape_ls.append(mape)
    r2_ls.append(r2)

    # Create residual plots
    plot_residuals(y_val, y_val_pred)

## Hyperparameter Tuning  
Based on baseline model training, the following models outperformed the other candidates across evaluation metrics (RMSE, MAPE, R-squared) on the validation data and were selected for hyperparameter tuning:
- Model 1 (e.g., Random Forest Regressor)  
  *Justification example*: Delivered low RMSE and MAPE scores during baseline evaluation, showing strong predictive performance with minimal error.
- Model 2 (e.g., XGBoost Regressor)  
  *Justification example*: Achieved high R-squared and consistently low error metrics, indicating its ability to capture underlying patterns in the data effectively.

### Random Forest Regressor  
Hyperparameters and Default Values:
- Model Complexity:
  - `n_estimators=100`: Number of trees in the forest.  
  - `max_depth=None`: Maximum depth of each tree; `None` allows trees to grow until all leaves are pure or minimum samples are reached.  
  - `min_samples_split=2`: Minimum number of samples required to split a node.  
  - `min_samples_leaf=1`: Minimum number of samples required at a leaf node.  
  - `max_features="auto"`: Number of features considered for the best split; default `auto` uses the square root of all features.  
- Regularization:
  - `max_leaf_nodes=None`: Maximum number of leaf nodes per tree.  
  - `min_impurity_decrease=0.0`: Splits a node only if it decreases impurity by this threshold.  
- Training Behavior:
  - `bootstrap=True`: Whether to use bootstrap samples for training each tree.  
  - `oob_score=False`: Whether to use out-of-bag samples to estimate prediction accuracy.  
  - `n_jobs=None`: Number of CPU threads used (`-1` for all processors).  
  - `random_state=None`: Random seed for reproducibility.  
  - `verbose=0`: Controls the verbosity of output during training.  
- Performance Optimization:
  - `max_samples=None`: Maximum number of samples used to train each tree, useful for subsampling large datasets.

For more details, refer to the official [scikit-learn RandomForestRegressor documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor).  

The following hyperparameters are typically the most impactful:
- `n_estimators`
- `max_depth`  
- `min_samples_split`
- `min_samples_leaf`
- `max_features`

In [ ]:
# Initialize model
rf = RandomForestRegressor(random_state=42)

# Define hyperparameter grid 
rf_param_grid = {
    "n_estimators": [100, 200, 500],               
    "max_depth": [None, 10, 20],              
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": [0.33, 0.66, 1]                
}

# Initialize grid search object
rf_grid_search = GridSearchCV(
    estimator=rf, 
    param_grid=rf_param_grid, 
    cv=5, 
    scoring="neg_root_mean_squared_error"  # use "r2" for R-squared or a custom function for MAPE
)

# Fit the grid search to the training data
rf_grid_search.fit(X_train_transformed, y_train)

In [ ]:
# DataFrame of grid search results 
rf_grid_search_results = pd.DataFrame({
    "validation_rmse": -1 * rf_grid_search.cv_results_["mean_test_score"],  # RMSE on validation data
    "parameters": rf_grid_search.cv_results_["params"]  # parameter values
}) 

# Extract each parameter as a separate column
rf_grid_search_results["n_estimators"] = rf_grid_search_results["parameters"].apply(lambda x: x["n_estimators"])
rf_grid_search_results["max_depth"] = rf_grid_search_results["parameters"].apply(lambda x: x["max_depth"])
rf_grid_search_results["min_samples_split"] = rf_grid_search_results["parameters"].apply(lambda x: x["min_samples_split"])
rf_grid_search_results["min_samples_leaf"] = rf_grid_search_results["parameters"].apply(lambda x: x["min_samples_leaf"])
rf_grid_search_results["max_features"] = rf_grid_search_results["parameters"].apply(lambda x: x["max_features"])

# Delete the parameters column
rf_grid_search_results = rf_grid_search_results.drop("parameters", axis=1)

# Show the top 10 best performing models
rf_grid_search_results.sort_values("validation_rmse")[:10]

### XGBoost Regressor  
Hyperparameters and Default Values:
- Model Complexity:  
    - `n_estimators=100`: Number of trees.  
    - `max_depth=6`: Maximum depth of each tree.  
    - `learning_rate=0.3`: Step size shrinking to prevent overfitting.  
    - `subsample=1.0`: Fraction of training samples used for each tree.  
    - `colsample_bytree=1.0`: Fraction of features used for each tree.  
    - `colsample_bylevel=1.0`: Fraction of features used at each tree level.  
    - `colsample_bynode=1.0`: Fraction of features used at each node.  
- Regularization and Learning:  
    - `gamma=0`: Minimum loss reduction required to make a further partition on a leaf node.  
    - `min_child_weight=1`: Minimum sum of instance weight (hessian) in a child.  
    - `scale_pos_weight=1`: Controls the balance of positive and negative weights; used for imbalanced datasets.  
- Training Behavior:  
    - `objective="reg:squarederror"`: Objective function for regression; default is for squared error.  
    - `booster="gbtree"`: Booster type; options include `"gbtree"` (default), `"gblinear"`, and `"dart"`.  
    - `tree_method="auto"`: Tree construction algorithm; `"auto"` chooses based on system configuration. Options include `"exact"`, `"approx"`, and `"hist"`.  
    - `eval_metric="rmse"`: Metric used for validation during training; default is root mean square error (`rmse`).  
- Performance Optimization:  
    - `early_stopping_rounds=None`: Stops training if validation metric does not improve after specified rounds.  
    - `n_jobs=1`: Number of threads used for parallel computation (`-1` for all processors).  
    - `random_state=None`: Seed for reproducibility.  
    - `verbose=1`: Verbosity level for training output; 0 for silent, higher values show more details.  
- Advanced Parameters:  
    - `lambda=1`: L2 regularization term on weights.  
    - `alpha=0`: L1 regularization term on weights.  
    - `max_delta_step=0`: Used to help with convergence in highly imbalanced datasets.

For more details, refer to the official [XGBoost documentation](https://xgboost.readthedocs.io/en/latest/parameter.html).

The following hyperparameters are typically the most impactful:
- `n_estimators`
- `max_depth` 
- `learning_rate`
- `subsample` 
- `colsample_bytree`
- `gamma`
- `min_child_weight`

In [ ]:
# Initialize model
xgb = XGBRegressor(random_state=42)

# Define hyperparameter grid 
xgb_param_grid = {
    "n_estimators": [100, 200, 300, 500],               
    "max_depth": [3, 6, 10], 
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],  
    "gamma": [0, 0.1, 0.2],           
    "min_child_weight": [1, 3, 5]
}

# Initialize grid search object
xgb_grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=xgb_param_grid, 
    cv=5, 
    scoring="neg_root_mean_squared_error"  # use "r2" for R-squared or a custom function for MAPE
)

# Fit the grid search to the training data
xgb_grid_search.fit(X_train_transformed, y_train)

In [ ]:
# DataFrame of grid search results 
xgb_grid_search_results = pd.DataFrame({
    "validation_rmse": -1 * xgb_grid_search.cv_results_["mean_test_score"],  # RMSE on validation data
    "parameters": xgb_grid_search.cv_results_["params"]  # parameter values
}) 

# Extract each parameter as a separate column
xgb_grid_search_results["n_estimators"] = xgb_grid_search_results["parameters"].apply(lambda x: x["n_estimators"])
xgb_grid_search_results["max_depth"] = xgb_grid_search_results["parameters"].apply(lambda x: x["max_depth"])
xgb_grid_search_results["learning_rate"] = xgb_grid_search_results["parameters"].apply(lambda x: x["learning_rate"])
xgb_grid_search_results["subsample"] = xgb_grid_search_results["parameters"].apply(lambda x: x["subsample"])
xgb_grid_search_results["colsample_bytree"] = xgb_grid_search_results["parameters"].apply(lambda x: x["colsample_bytree"])
xgb_grid_search_results["gamma"] = xgb_grid_search_results["parameters"].apply(lambda x: x["gamma"])
xgb_grid_search_results["min_child_weight"] = xgb_grid_search_results["parameters"].apply(lambda x: x["min_child_weight"])

# Delete the parameters column
xgb_grid_search_results = xgb_grid_search_results.drop("parameters", axis=1)

# Show the top 10 best performing models
xgb_grid_search_results.sort_values("validation_rmse")[:10]

## Final Model  
Based on hyperparameter tuning, the following model achieved the best performance (e.g., RMSE = 12.34) on the validation data compared to other candidates, making it the optimal choice for the final model. This model will be further evaluated on the test data to confirm its generalizability.

Example final model: XGBoost Regressor  
Selected hyperparameters:
- `n_estimators=300`
- `max_depth=4` 
- `learning_rate=0.1`
- `subsample=0.8` 
- `colsample_bytree=0.8`
- `gamma=0.1`
- `min_child_weight=3`
- `random_state=42`

### Training Final Model

In [ ]:
# Initialize model
final_model = XGBRegressor(
    n_estimators=300, 
    max_depth=4, 
    learning_rate=0.1,
    subsample=0.8, 
    colsample_bytree=0.8, 
    gamma=0.1,
    min_child_weight=3, 
    random_state=42
)

# Train model
final_model.fit(X_train_transformed, y_train)

### Model Evaluation

In [ ]:
# Predict on the training, validation and test data
y_train_pred = final_model.predict(X_train_transformed)
y_val_pred = final_model.predict(X_val_transformed)
y_test_pred = final_model.predict(X_test_transformed)

# Evaluate model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Create table of evaluation metrics
final_model_evaluation = pd.DataFrame({
    "Metric": ["RMSE", "MAPE", "R-squared"],
    "Training": [train_rmse, train_mape, train_r2],
    "Validation": [val_rmse, val_mape, val_r2],
    "Test": [test_rmse, test_mape, test_r2]
})

# Show evaluation metrics
print(final_model_evaluation.round(2))  # round metrics to 2 decimals

In [ ]:
# Resiudal plots for validation data
plot_residuals(y_val, y_val_pred)

In [ ]:
# Resiudal plots for tets data
plot_residuals(y_test, y_test_pred)

### Feature Importance 

XGBoost Feature Importance Plot:

In [ ]:
# Mapping of feature names for better readability
feature_mapping = {
    "f0": "feature_1",
    "f1": "feature_2",
    "f2": "feature_3",
    "f3": "feature_4",
    "f4": "feature_5",
    "f5": "feature_6",
    "f6": "feature_7",
    "f7": "feature_8",
    "f8": "feature_9",
    "f9": "feature_10"
}

# Create feature importance plot of the top 10 features
# Note: final_model must be an XGBoost model
xgb.plot_importance(final_model, max_num_features=10)

# Map the feature names
plt.gca().set_yticklabels([feature_mapping.get(label.get_text(), label.get_text()) 
                           for label in plt.gca().get_yticklabels()])

# Show the plot
plt.show()

### Saving Model  
Save both the column transformer and the final model as pickle files in the `models` directory for later use.

In [ ]:
# Create models directory if it doesn't exist
os.makedirs("models", exist_ok=True)

# Save column transformer 
try:
    with open("models/column_transformer.pkl", "wb") as file:
        pickle.dump(column_transformer, file)
    print("Column transformer saved successfully")
except Exception as e:
    print(f"An error occurred while saving the column transformer: {e}}")
    
# Save final model
try:
    with open("models/final_model.pkl", "wb") as file:
        pickle.dump(final_model, file)
    print("Final model saved successfully")
except Exception as e:
    print(f"An error occurred while saving the final model: {e}}")

# Modeling (Classification Task)  
For a classification problem, where the goal is to predict a categorical target variable.

## Training Baseline Models (Individually) 

### Logistic Regression  
Hyperparameters and Default Values:
- `penalty="l2"`: Regularization type (`"l1"`, `"l2"`, `"elasticnet"`, or `"none"`).
- `C=1.0`: Inverse of regularization strength (smaller = stronger regularization).
- `solver="lbfgs"`: Optimization algorithm (`"lbfgs"`, `"liblinear"`, `"saga"`, etc.).
- `max_iter=100`: Maximum iterations for convergence.
- `multi_class="auto"`: Multi-class handling (`"ovr"` or `"multinomial"`).
- `fit_intercept=True`: Whether to calculate the intercept.
- `class_weight=None`: Class weights; `"balanced"` adjusts for imbalance.

For more details, refer to the official [scikit-learn LogisticRegression documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [ ]:
# Initialize model
lr = LogisticRegression()

# Train model
lr.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = lr.predict(X_train_transformed)
y_val_pred = lr.predict(X_val_transformed)

# Evaluate model: Classification report
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Evaluate model: Confusion matrix 
print("Training Confusion Matrix:")
cm_train = confusion_matrix(y_train, y_train_pred)
cm_disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=lr.classes_)
cm_disp_train.plot(cmap="Blues")
plt.title("Training Confusion Matrix")
plt.show()b

print("Validation Confusion Matrix:")
cm_val = confusion_matrix(y_val, y_val_pred)
cm_disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=lr.classes_)
cm_disp_val.plot(cmap="Blues")
plt.title("Validation Confusion Matrix")
plt.show()

### Support Vector Classifier  
Hyperparameters and Default Values:
- `C=1.0`: Regularization strength (smaller = stronger regularization).
- `kernel="rbf"`: Kernel type (`"linear"`, `"poly"`, `"rbf"`, `"sigmoid"`, or callable).
- `degree=3`: Degree of the polynomial kernel (ignored for other kernels).
- `gamma="scale"`: Kernel coefficient (`"scale"`, `"auto"`, or float).
- `coef0=0.0`: Independent term in kernel functions (`"poly"` and `"sigmoid"` only).
- `class_weight=None`: Class weights; `"balanced"` adjusts for imbalance.
- `max_iter=-1`: Maximum iterations (unlimited if `-1`).
- `probability=False`: Enables probability estimates (slower training).
- `random_state=None`: Seed for reproducibility (affects `shrinking` and `probability`).

For more details, refer to the official [scikit-learn SVC documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

In [ ]:
# Initialize model
svc = SVC()  

# Train model
svc.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = svc.predict(X_train_transformed)
y_val_pred = svc.predict(X_val_transformed)

# Evaluate model: Classification report
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Evaluate model: Confusion matrix
print("Training Confusion Matrix:")
cm_train = confusion_matrix(y_train, y_train_pred)
cm_disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=svc.classes_)
cm_disp_train.plot(cmap="Blues")
plt.title("Training Confusion Matrix")
plt.show()

print("Validation Confusion Matrix:")
cm_val = confusion_matrix(y_val, y_val_pred)
cm_disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=svc.classes_)
cm_disp_val.plot(cmap="Blues")
plt.title("Validation Confusion Matrix")
plt.show()

### Random Forest Classifier  
Hyperparameters and Default Values:
- Model Complexity:
  - `n_estimators=100`: Number of trees in the forest.
  - `max_depth=None`: Maximum depth of each tree; `None` allows trees to grow until all leaves are pure or minimum samples are reached.
  - `min_samples_split=2`: Minimum number of samples required to split a node.
  - `min_samples_leaf=1`: Minimum number of samples required at a leaf node.
  - `max_features="auto"`: Number of features considered for the best split; default `auto` uses the square root of all features.
- Regularization:
  - `max_leaf_nodes=None`: Maximum number of leaf nodes per tree.
  - `min_impurity_decrease=0.0`: Splits a node only if it decreases impurity by this threshold.
  - `class_weight=None`: Weights associated with classes. If `None`, all classes are supposed to have weight one. Use `"balanced"` to automatically adjust weights inversely proportional to class frequencies in the input data.
- Training Behavior:
  - `bootstrap=True`: Whether to use bootstrap samples for training each tree.
  - `oob_score=False`: Whether to use out-of-bag samples to estimate prediction accuracy.
  - `n_jobs=None`: Number of CPU threads used (`-1` for all processors).
  - `random_state=None`: Random seed for reproducibility.
  - `verbose=0`: Controls the verbosity of output during training.
- Performance Optimization:
  - `max_samples=None`: Maximum number of samples used to train each tree, useful for subsampling large datasets.

For more details, refer to the official [scikit-learn RandomForestClassifier documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier).


In [ ]:
# Initialize model
rf = RandomForestClassifier(random_state=42)

# Train model
rf.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = rf.predict(X_train_transformed)
y_val_pred = rf.predict(X_val_transformed)

# Evaluate model: Classification report
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Evaluate model: Confusion matrix 
print("Training Confusion Matrix:")
cm_train = confusion_matrix(y_train, y_train_pred)
cm_disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=rf.classes_)
cm_disp_train.plot(cmap="Blues")
plt.title("Training Confusion Matrix")
plt.show()

print("Validation Confusion Matrix:")
cm_val = confusion_matrix(y_val, y_val_pred)
cm_disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=rf.classes_)
cm_disp_val.plot(cmap="Blues")
plt.title("Validation Confusion Matrix")
plt.show()

### Multi-Layer Perceptron Classifier  
Hyperparameters and Default Values:
- Model Architecture
    - `hidden_layer_sizes=(100,)`: Number and size of hidden layers; e.g., `(100,)` = 1 layer with 100 neurons.
    - `activation="relu"`: Activation function; options: `"relu"`, `"tanh"`, `"logistic"`, `"identity"`.
- Optimization
    - `solver="adam"`: Optimization algorithm; options: `"adam"`, `"lbfgs"`, `"sgd"`.
    - `alpha=0.0001`: L2 regularization to reduce overfitting.
    - `learning_rate="constant"`: Learning rate strategy; options: `"constant"`, `"invscaling"`, `"adaptive"`.
    - `learning_rate_init=0.001`: Initial learning rate.
    - `power_t=0.5`: Used for `"invscaling"` learning rate.
- Training Behavior
    - `max_iter=200`: Maximum training iterations.
    - `tol=1e-4`: Stopping criterion for improvement tolerance.
    - `momentum=0.9`: Momentum for SGD (used with `solver="sgd"`).
    - `early_stopping=False`: Stop early if no improvement on validation set.
    - `n_iter_no_change=10`: Iterations without improvement to stop training.
- Performance
    - `batch_size="auto"`: Batch size; `"auto"` = `min(200, n_samples)`.
    - `shuffle=True`: Shuffle training data every epoch.
    - `random_state=None`: Seed for reproducibility.
    - `verbose=False`: Control output verbosity.
    - `warm_start=False`: Retain model state for further training.
- Classification-Specific
    - `validation_fraction=0.1`: Fraction of data for validation (used with `early_stopping=True`).
    - `out_activation_="softmax"`: Output activation for multi-class classification.

For more details, refer to the official [scikit-learn MLPClassifier documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html).


In [ ]:
# Initialize model
mlp = MLPClassifier(random_state=42)

# Train model
mlp.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = mlp.predict(X_train_transformed)
y_val_pred = mlp.predict(X_val_transformed)

# Evaluate model: Classification report
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Evaluate model: Confusion matrix
print("Training Confusion Matrix:")
cm_train = confusion_matrix(y_train, y_train_pred)
cm_disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=mlp.classes_)
cm_disp_train.plot(cmap="Blues")
plt.title("Training Confusion Matrix")
plt.show()

print("Validation Confusion Matrix:")
cm_val = confusion_matrix(y_val, y_val_pred)
cm_disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=mlp.classes_)
cm_disp_val.plot(cmap="Blues")
plt.title("Validation Confusion Matrix")
plt.show()

### XGBoost Classifier  
Hyperparameters and Default Values:
- Model Complexity:  
    - `n_estimators=100`: Number of trees (boosting rounds).  
    - `max_depth=6`: Maximum depth of each tree.  
    - `learning_rate=0.3`: Step size shrinkage to prevent overfitting.  
    - `subsample=1.0`: Fraction of training samples used per tree.  
    - `colsample_bytree=1.0`: Fraction of features used per tree.  
    - `colsample_bylevel=1.0`: Fraction of features used per tree level.  
    - `colsample_bynode=1.0`: Fraction of features used per split node.  
- Regularization and Learning:  
    - `gamma=0`: Minimum loss reduction required to split a leaf node.  
    - `min_child_weight=1`: Minimum sum of instance weights (hessian) in a child.  
    - `scale_pos_weight=1`: Balances positive and negative class weights for imbalanced datasets.  
- Training Behavior:  
    - `objective="binary:logistic"`: Objective function for binary classification; alternatives include `"multi:softmax"` or `"multi:softprob"`.  
    - `booster="gbtree"`: Booster type; options include `"gbtree"` (default), `"gblinear"`, and `"dart"`.  
    - `tree_method="auto"`: Tree construction algorithm; options: `"exact"`, `"approx"`, `"hist"`, `"gpu_hist"`.  
    - `eval_metric="logloss"`: Default evaluation metric for binary classification. Options include `"error"`, `"auc"`, and others.  
- Performance Optimization:  
    - `early_stopping_rounds=None`: Stops training if validation metric does not improve after specified rounds.  
    - `n_jobs=1`: Number of threads for parallel computation (`-1` for all processors).  
    - `random_state=None`: Seed for reproducibility.  
    - `verbose=1`: Verbosity level; 0 for silent, higher values for detailed output.  
- Advanced Parameters:  
    - `lambda=1`: L2 regularization term on weights.  
    - `alpha=0`: L1 regularization term on weights.  
    - `max_delta_step=0`: Helps with convergence in imbalanced datasets.

For more details, refer to the official [XGBoost documentation](https://xgboost.readthedocs.io/en/latest/parameter.html).


In [ ]:
# Initialize model
xgb = XGBClassifier(random_state=42)

# Train model
xgb.fit(X_train_transformed, y_train)

# Predict on the training and validation data
y_train_pred = xgb.predict(X_train_transformed)
y_val_pred = xgb.predict(X_val_transformed)

# Evaluate model: Classification report
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Evaluate model: Confusion matrix 
print("Training Confusion Matrix:")
cm_train = confusion_matrix(y_train, y_train_pred)
cm_disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=xgb.classes_)
cm_disp_train.plot(cmap="Blues")
plt.title("Training Confusion Matrix")
plt.show()

print("Validation Confusion Matrix:")
cm_val = confusion_matrix(y_val, y_val_pred)
cm_disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=xgb.classes_)
cm_disp_val.plot(cmap="Blues")
plt.title("Validation Confusion Matrix")
plt.show()

## Training Baseline Models (Pipeline)  
Train five baseline models:
- Logistic Regression
- Support Vector Classifier
- Random Forest Classifier
- Multi-Layer Perceptron Classifier
- XGBoost Classifier

The model performance will be evaluated using the following metrics:  
- Accuracy
- Precision
- Recall
- F1 score

In [ ]:
# Define models with baseline configurations
models = [LogisticRegression(), SVC(), RandomForestClassifier(random_state=42), MLPClassifier(random_state=42), XGBClassifier(random_state=42)]

# Loop through each model
for model in models:
    # Show model
    print("=" * 100)
    print(f"Model: {model}")

    # Scale numerical columns and encode categorical columns 
    column_transformer = ColumnTransformer(
        transformers=[
            ("scaler", StandardScaler(), numerical_columns),
            ("encoder", OneHotEncoder(drop="first"), categorical_columns)
        ],
        remainder="passthrough"  # Include the boolean columns without transformation
    )

    # Create a pipeline
    pipeline = Pipeline(steps=[
        ("column_transformer", column_transformer),
        ("model", model)
    ])

    # Fit the pipeline on the training data
    pipeline.fit(X_train, y_train)
    
    # Predict on the validation data
    y_val_pred = pipeline.predict(X_val)

    # Evaluate model: Classification report
    print("Classification Report:")
    print(classification_report(y_val, y_val_pred))
    
    # Evaluate model: Confusion matrix
    print("Confusion Matrix:")
    cm_val = confusion_matrix(y_val, y_val_pred)
    cm_disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val)
    cm_disp_val.plot(cmap="Blues")
    plt.title("Confusion Matrix")
    plt.show()

## Hyperparameter Tuning  
Based on baseline model training, the following models outperformed the other candidates on evaluation metrics such as accuracy, recall, precision, and F1 score on the validation data and were selected for hyperparameter tuning:
- Model 1 (e.g., Random Forest Classifier)  
  *Justification example*: Demonstrated high recall and F1 score, indicating robust performance across multiple classes and strong handling of imbalanced datasets. 
- Model 2 (e.g., XGBoost Classifier)  
  *Justification example*: Consistently high accuracy across classes and strong overall evaluation metric scores, making it a competitive option for fine-tuning.

### Random Forest Classifier  
Hyperparameters and Default Values:
- Model Complexity:
  - `n_estimators=100`: Number of trees in the forest.
  - `max_depth=None`: Maximum depth of each tree; `None` allows trees to grow until all leaves are pure or minimum samples are reached.
  - `min_samples_split=2`: Minimum number of samples required to split a node.
  - `min_samples_leaf=1`: Minimum number of samples required at a leaf node.
  - `max_features="auto"`: Number of features considered for the best split; default `auto` uses the square root of all features.
- Regularization:
  - `max_leaf_nodes=None`: Maximum number of leaf nodes per tree.
  - `min_impurity_decrease=0.0`: Splits a node only if it decreases impurity by this threshold.
  - `class_weight=None`: Weights associated with classes. If `None`, all classes are supposed to have weight one. Use `"balanced"` to automatically adjust weights inversely proportional to class frequencies in the input data.
- Training Behavior:
  - `bootstrap=True`: Whether to use bootstrap samples for training each tree.
  - `oob_score=False`: Whether to use out-of-bag samples to estimate prediction accuracy.
  - `n_jobs=None`: Number of CPU threads used (`-1` for all processors).
  - `random_state=None`: Random seed for reproducibility.
  - `verbose=0`: Controls the verbosity of output during training.
- Performance Optimization:
  - `max_samples=None`: Maximum number of samples used to train each tree, useful for subsampling large datasets.

For more details, refer to the official [scikit-learn RandomForestClassifier documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier).

The following hyperparameters are typically the most impactful:
- `n_estimators`
- `max_depth`  
- `min_samples_split`
- `min_samples_leaf`
- `max_features`

In [ ]:
# Initialize model
rf = RandomForestClassifier(random_state=42)

# Define hyperparameter grid 
rf_param_grid = {
    "n_estimators": [100, 200, 500],               
    "max_depth": [None, 10, 20],              
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": [0.33, 0.66, 1]                
}

# Initialize grid search object
rf_grid_search = GridSearchCV(
    estimator=rf, 
    param_grid=rf_param_grid, 
    cv=5, 
    scoring="accuracy"  # use "f1", "precision", "recall" or "roc_auc" optionally
)

# Fit the grid search to the training data
rf_grid_search.fit(X_train_transformed, y_train)

In [ ]:
# DataFrame of grid search results 
rf_grid_search_results = pd.DataFrame({
    "validation_accuracy": rf_grid_search.cv_results_["mean_test_score"],  # accuracy on validation data
    "parameters": rf_grid_search.cv_results_["params"]  # parameter values
}) 

# Extract each parameter as a separate column
rf_grid_search_results["n_estimators"] = rf_grid_search_results["parameters"].apply(lambda x: x["n_estimators"])
rf_grid_search_results["max_depth"] = rf_grid_search_results["parameters"].apply(lambda x: x["max_depth"])
rf_grid_search_results["min_samples_split"] = rf_grid_search_results["parameters"].apply(lambda x: x["min_samples_split"])
rf_grid_search_results["min_samples_leaf"] = rf_grid_search_results["parameters"].apply(lambda x: x["min_samples_leaf"])
rf_grid_search_results["max_features"] = rf_grid_search_results["parameters"].apply(lambda x: x["max_features"])

# Delete the parameters column
rf_grid_search_results = rf_grid_search_results.drop("parameters", axis=1)

# Show the top 10 best performing models
print(rf_grid_search_results.sort_values("validation_accuracy", ascending=False).head(10))

### XGBoost Classifier  
Hyperparameters and Default Values:
- Model Complexity:  
    - `n_estimators=100`: Number of trees (boosting rounds).  
    - `max_depth=6`: Maximum depth of each tree.  
    - `learning_rate=0.3`: Step size shrinkage to prevent overfitting.  
    - `subsample=1.0`: Fraction of training samples used per tree.  
    - `colsample_bytree=1.0`: Fraction of features used per tree.  
    - `colsample_bylevel=1.0`: Fraction of features used per tree level.  
    - `colsample_bynode=1.0`: Fraction of features used per split node.  
- Regularization and Learning:  
    - `gamma=0`: Minimum loss reduction required to split a leaf node.  
    - `min_child_weight=1`: Minimum sum of instance weights (hessian) in a child.  
    - `scale_pos_weight=1`: Balances positive and negative class weights for imbalanced datasets.  
- Training Behavior:  
    - `objective="binary:logistic"`: Objective function for binary classification; alternatives include `"multi:softmax"` or `"multi:softprob"`.  
    - `booster="gbtree"`: Booster type; options include `"gbtree"` (default), `"gblinear"`, and `"dart"`.  
    - `tree_method="auto"`: Tree construction algorithm; options: `"exact"`, `"approx"`, `"hist"`, `"gpu_hist"`.  
    - `eval_metric="logloss"`: Default evaluation metric for binary classification. Options include `"error"`, `"auc"`, and others.  
- Performance Optimization:  
    - `early_stopping_rounds=None`: Stops training if validation metric does not improve after specified rounds.  
    - `n_jobs=1`: Number of threads for parallel computation (`-1` for all processors).  
    - `random_state=None`: Seed for reproducibility.  
    - `verbose=1`: Verbosity level; 0 for silent, higher values for detailed output.  
- Advanced Parameters:  
    - `lambda=1`: L2 regularization term on weights.  
    - `alpha=0`: L1 regularization term on weights.  
    - `max_delta_step=0`: Helps with convergence in imbalanced datasets.

For more details, refer to the official [XGBoost documentation](https://xgboost.readthedocs.io/en/latest/parameter.html).

The following hyperparameters are typically the most impactful:
- `n_estimators`
- `max_depth` 
- `learning_rate`
- `subsample` 
- `colsample_bytree`
- `gamma`
- `min_child_weight`

In [ ]:
# Initialize model
xgb = XGBClassifier(random_state=42)

# Define hyperparameter grid 
xgb_param_grid = {
    "n_estimators": [100, 200, 300, 500],               
    "max_depth": [3, 6, 10], 
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],  
    "gamma": [0, 0.1, 0.2],           
    "min_child_weight": [1, 3, 5]
}

# Initialize grid search object
xgb_grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=xgb_param_grid, 
    cv=5, 
    scoring="accuracy"  # use "f1", "precision", "recall" or "roc_auc" optionally
)

# Fit the grid search to the training data
xgb_grid_search.fit(X_train_transformed, y_train)

In [ ]:
# DataFrame of grid search results 
xgb_grid_search_results = pd.DataFrame({
    "validation_accuracy": xgb_grid_search.cv_results_["mean_test_score"],  # accuracy on validation data
    "parameters": xgb_grid_search.cv_results_["params"]  # parameter values
}) 

# Extract each parameter as a separate column
xgb_grid_search_results["n_estimators"] = xgb_grid_search_results["parameters"].apply(lambda x: x["n_estimators"])
xgb_grid_search_results["max_depth"] = xgb_grid_search_results["parameters"].apply(lambda x: x["max_depth"])
xgb_grid_search_results["learning_rate"] = xgb_grid_search_results["parameters"].apply(lambda x: x["learning_rate"])
xgb_grid_search_results["subsample"] = xgb_grid_search_results["parameters"].apply(lambda x: x["subsample"])
xgb_grid_search_results["colsample_bytree"] = xgb_grid_search_results["parameters"].apply(lambda x: x["colsample_bytree"])
xgb_grid_search_results["gamma"] = xgb_grid_search_results["parameters"].apply(lambda x: x["gamma"])
xgb_grid_search_results["min_child_weight"] = xgb_grid_search_results["parameters"].apply(lambda x: x["min_child_weight"])

# Delete the parameters column
xgb_grid_search_results = xgb_grid_search_results.drop("parameters", axis=1)

# Show the top 10 best performing models
print(xgb_grid_search_results.sort_values("validation_accuracy", ascending=False).head(10))

## Final Model  
Based on hyperparameter tuning, the following model achieved the best performance (e.g., accuracy = 0.92) on the validation data compared to other candidates, making it the optimal choice for the final model. This model will be further evaluated on the test data to confirm its generalizability.

Example final model: XGBoost Classifier    
Selected hyperparameters:
- `n_estimators=300`
- `max_depth=4` 
- `learning_rate=0.1`
- `subsample=0.8` 
- `colsample_bytree=0.8`
- `gamma=0.1`
- `min_child_weight=3`
- `random_state=42`

### Training Final Model

In [ ]:
# Initialize model
final_model = XGBClassifier(
    n_estimators=300, 
    max_depth=4, 
    learning_rate=0.1,
    subsample=0.8, 
    colsample_bytree=0.8, 
    gamma=0.1,
    min_child_weight=3, 
    random_state=42
)

# Train model
final_model.fit(X_train_transformed, y_train)

### Model Evaluation

In [ ]:
# Predict on the training, validation and test data
y_train_pred = final_model.predict(X_train_transformed)
y_val_pred = final_model.predict(X_val_transformed)
y_test_pred = final_model.predict(X_test_transformed)

# Evaluate model: Classification report
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

# Evaluate model: Confusion matrix
print("Training Confusion Matrix:")
cm_train = confusion_matrix(y_train, y_train_pred)
cm_disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=final_model.classes_)
cm_disp_train.plot(cmap="Blues")
plt.title("Training Confusion Matrix")
plt.show()

print("Validation Confusion Matrix:")
cm_val = confusion_matrix(y_val, y_val_pred)
cm_disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=final_model.classes_)
cm_disp_val.plot(cmap="Blues")
plt.title("Validation Confusion Matrix")
plt.show()

print("Test Confusion Matrix:")
cm_test = confusion_matrix(y_test, y_test_pred)
cm_disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=final_model.classes_)
cm_disp_test.plot(cmap="Blues")
plt.title("Test Confusion Matrix")
plt.show()

### Feature Importance 

XGBoost Feature Importance Plot:

In [ ]:
# Mapping of feature names for better readability
feature_mapping = {
    "f0": "feature_1",
    "f1": "feature_2",
    "f2": "feature_3",
    "f3": "feature_4",
    "f4": "feature_5",
    "f5": "feature_6",
    "f6": "feature_7",
    "f7": "feature_8",
    "f8": "feature_9",
    "f9": "feature_10"
}

# Create feature importance plot of the top 10 features
# Note: final_model must be an XGBoost model
xgb.plot_importance(final_model, max_num_features=10)

# Map the feature names
plt.gca().set_yticklabels([feature_mapping.get(label.get_text(), label.get_text()) 
                           for label in plt.gca().get_yticklabels()])

# Show the plot
plt.show()

### Saving Model  
Save both the column transformer and the final model as pickle files in the `models` directory for later use.

In [ ]:
# Create models directory if it doesn't exist
os.makedirs("models", exist_ok=True)

# Save column transformer 
try:
    with open("models/column_transformer.pkl", "wb") as file:
        pickle.dump(column_transformer, file)
    print("Column transformer saved successfully")
except Exception as e:
    print(f"An error occurred while saving the column transformer: {e}}")
    
# Save final model
try:
    with open("models/final_model.pkl", "wb") as file:
        pickle.dump(final_model, file)
    print("Final model saved successfully")
except Exception as e:
    print(f"An error occurred while saving the final model: {e}}")

# Summary

**Data Preprocessing**:
- Removed duplicates (e.g., based on the ID column) using `drop_duplicates` from `pandas`.
- Handled incorrect data types (e.g., converted string columns to numerical or datetime columns) using `astype` or `to_datetime` from `pandas`.
- Extracted features (e.g., created categorical, numerical, or boolean features from string columns) using custom functions with `apply` from `pandas`, `lambda` functions, and `re` for pattern matching.
- Handled missing values (e.g., through deletion, median imputation for numerical columns, or mode imputation for categorical columns) using `dropna` or `fillna` from `pandas`.
- Handled outliers (e.g., removed them using the 3SD method or 1.5 IQR method) with a custom transformer class utilizing `BaseEstimator` and `TransformerMixin` from `sklearn`.
- Saved the preprocessed data as a .csv file using `to_csv` from `pandas` or in a MySQL database table using `sqlalchemy`, `mysql-connector-python`, and `to_sql` from `pandas`.
- Split data into training (e.g., 70%), validation (15%), and test (15%) sets using `train_test_split` from `sklearn`.
- Scaled numerical features (e.g., using standard scaling or min-max normalization) with `StandardScaler` or `MinMaxScaler` from `sklearn`.
- Encoded categorical features (e.g., using one-hot encoding) with `OneHotEncoder` from `sklearn`.

**Exploratory Data Analysis (EDA)**:
- Analyzed descriptive statistics (e.g., mean, median, standard deviation) of numerical columns and visualized distributions (e.g., using histograms).
- Examined frequencies of categorical columns and visualized them (e.g., using bar plots or a bar plot matrix).
- Assessed bivariate relationships between numerical columns (e.g., using a correlation heatmap, scatterplots, or a scatterplot matrix).
- Explored relationships between numerical and categorical columns with group-wise statistics (e.g., mean or median by category) and visualized them (e.g., through bar plots or a bar plot matrix).

**Modeling**:
- Trained baseline models to establish performance benchmarks:
    - For a regression task (e.g., Linear Regression, Support Vector Regressor, Random Forest Regressor, Multi-Layer Perceptron Regressor, XGBoost Regressor).
    - For a classification task (e.g., Logistic Regression, Support Vector Classifier, Random Forest Classifier, Multi-Layer Perceptron Classifier, XGBoost Classifier).
- Performed hyperparameter tuning (e.g., using grid search).
- Selected the final model based on performance evaluation:
    - For a regression task (e.g., using RMSE, MAPE, or R-squared)
    - For a classification task (e.g., using accuracy, precision, recall, or F1 score).
- Saved the final model (e.g., as a .pkl file using `pickle`).

**Next Steps**:
- Add K-Nearest Neighbors (KNN) as individual baseline model and in model pipeline for both regression and classification task. 
- Add Decision Tree as individual baseline model and in model pipeline for both regression and classification task.
- Add Lasso, Ridge, and Polynomial Regression for modeling.
- Add RandomSearchCV for hyperparameter tuning.
- Add unsupervised learning section with algorithms like K-Means Clustering, DBSCAN, and Principal Component Analysis (PCA).
- Add model deployment as a web application with an interactive UI and API (e.g., using Gradio or Flask).